# Transaction Amount Prediction using Regression Models

## Objective
This notebook implements an end-to-end machine learning **regression pipeline** to predict transaction amounts. We will:
1. Perform Exploratory Data Analysis (EDA)
2. Preprocess and clean the data
3. Engineer relevant features
4. Train multiple regression models
5. Evaluate and compare model performance using regression metrics

## Dataset
- **Source**: Credit card transaction data
- **Target**: `TransactionAmt` (continuous value - amount of transaction)
- **Features**: Various transaction, card, and customer-related features

## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

# Evaluation metrics
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    mean_absolute_percentage_error
)

# Set random seed for reproducibility
np.random.seed(42)

print("✓ All libraries imported successfully!")

## 2. Load and Explore the Dataset

In [ ]:
# Load the training data
train_df = pd.read_csv('datasets/transaction/train_transaction.csv')

print("Dataset Shape:", train_df.shape)
print("\n" + "="*80)
print("First 5 rows:")
print(train_df.head())
print("\n" + "="*80)
print("Target Variable Statistics (TransactionAmt):")
print(train_df['TransactionAmt'].describe())
print("\n" + "="*80)
print("Missing Values:")
print(f"Total missing values: {train_df.isnull().sum().sum()}")

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Visualize Transaction Amount Distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
axes[0].hist(train_df['TransactionAmt'], bins=100, color='skyblue', edgecolor='black')
axes[0].set_title('Transaction Amount Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Transaction Amount ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_xlim(0, 1000)  # Focus on common range

# Box plot
axes[1].boxplot(train_df['TransactionAmt'], vert=True)
axes[1].set_title('Transaction Amount Box Plot', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Transaction Amount ($)')
axes[1].set_ylim(0, 500)

# Log scale distribution
axes[2].hist(np.log1p(train_df['TransactionAmt']), bins=100, color='lightcoral', edgecolor='black')
axes[2].set_title('Log-Transformed Transaction Amount', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Log(Transaction Amount)')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"\nTransaction Amount Range: ${train_df['TransactionAmt'].min():.2f} - ${train_df['TransactionAmt'].max():.2f}")
print(f"Mean Transaction Amount: ${train_df['TransactionAmt'].mean():.2f}")
print(f"Median Transaction Amount: ${train_df['TransactionAmt'].median():.2f}")

## 4. Data Preprocessing and Feature Engineering

In [ ]:
# Create a working copy
df = train_df.copy()

# Drop TransactionID and isFraud (for regression, we don't use fraud label)
df = df.drop(['TransactionID', 'isFraud'], axis=1, errors='ignore')

# Select important features for regression
important_cols = ['TransactionAmt', 'TransactionDT', 'ProductCD',
                  'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
                  'addr1', 'addr2', 'dist1', 'dist2',
                  'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10',
                  'D1', 'D2', 'D3', 'D4', 'D5', 'D10', 'D15',
                  'M1', 'M2', 'M3', 'M4',
                  'V12', 'V13', 'V20', 'V36', 'V37', 'V45', 'V53', 'V54']

# Keep only columns that exist
important_cols = [col for col in important_cols if col in df.columns]
df = df[important_cols]

print(f"Selected {len(important_cols)} features for modeling")
print(f"Dataset shape after feature selection: {df.shape}")

# Handle categorical variables
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
if 'TransactionAmt' in categorical_cols:
    categorical_cols.remove('TransactionAmt')

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

print(f"\n✓ Categorical variables encoded")

# Handle missing values
print(f"\nMissing values before imputation: {df.isnull().sum().sum()}")
imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
print(f"Missing values after imputation: {df_imputed.isnull().sum().sum()}")

# Remove outliers using IQR method (optional - keeps extreme values reasonable)
Q1 = df_imputed['TransactionAmt'].quantile(0.25)
Q3 = df_imputed['TransactionAmt'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

df_clean = df_imputed[(df_imputed['TransactionAmt'] >= lower_bound) & 
                      (df_imputed['TransactionAmt'] <= upper_bound)]

print(f"\nData shape after outlier removal: {df_clean.shape}")
print(f"Removed {len(df_imputed) - len(df_clean)} outliers")

In [ ]:
# Separate features and target
X = df_clean.drop('TransactionAmt', axis=1)
y = df_clean['TransactionAmt']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Data split and scaled successfully")

## 5. Model Training and Evaluation

We will train and evaluate the following regression models:
1. **Linear Regression** - Simple baseline model
2. **Ridge Regression** - Linear with L2 regularization
3. **Lasso Regression** - Linear with L1 regularization
4. **Random Forest Regressor** - Ensemble method
5. **Gradient Boosting Regressor** - Advanced boosting
6. **XGBoost Regressor** - Optimized gradient boosting

In [ ]:
# Function to evaluate regression models
def evaluate_regression_model(name, model, X_train, y_train, X_test, y_test):
    """
    Train and evaluate a regression model
    """
    print(f"\n{'='*80}")
    print(f"Training {name}...")
    print('='*80)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    print(f"\n📊 Training Performance:")
    print(f"   R² Score: {train_r2:.4f}")
    print(f"   RMSE:     ${train_rmse:.2f}")
    print(f"   MAE:      ${train_mae:.2f}")
    
    print(f"\n📊 Testing Performance:")
    print(f"   R² Score: {test_r2:.4f}")
    print(f"   RMSE:     ${test_rmse:.2f}")
    print(f"   MAE:      ${test_mae:.2f}")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Predicted vs Actual
    axes[0].scatter(y_test, y_pred_test, alpha=0.5, s=10)
    axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[0].set_xlabel('Actual Transaction Amount ($)')
    axes[0].set_ylabel('Predicted Transaction Amount ($)')
    axes[0].set_title(f'{name} - Predicted vs Actual', fontweight='bold')
    axes[0].grid(alpha=0.3)
    
    # Residual plot
    residuals = y_test - y_pred_test
    axes[1].scatter(y_pred_test, residuals, alpha=0.5, s=10)
    axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
    axes[1].set_xlabel('Predicted Transaction Amount ($)')
    axes[1].set_ylabel('Residuals ($)')
    axes[1].set_title(f'{name} - Residual Plot', fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'model': name,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae
    }

print("✓ Evaluation function defined")

### 5.1 Linear Regression

In [ ]:
# Train Linear Regression
lr_model = LinearRegression()
lr_results = evaluate_regression_model('Linear Regression', lr_model,
                                       X_train_scaled, y_train,
                                       X_test_scaled, y_test)

### 5.2 Ridge Regression (L2 Regularization)

In [ ]:
# Train Ridge Regression
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_results = evaluate_regression_model('Ridge Regression', ridge_model,
                                         X_train_scaled, y_train,
                                         X_test_scaled, y_test)

### 5.3 Lasso Regression (L1 Regularization)

In [ ]:
# Train Lasso Regression
lasso_model = Lasso(alpha=0.1, random_state=42, max_iter=10000)
lasso_results = evaluate_regression_model('Lasso Regression', lasso_model,
                                         X_train_scaled, y_train,
                                         X_test_scaled, y_test)

### 5.4 Random Forest Regressor

In [ ]:
# Train Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
rf_results = evaluate_regression_model('Random Forest', rf_model,
                                      X_train_scaled, y_train,
                                      X_test_scaled, y_test)

# Feature Importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(data=feature_importance.head(15), x='importance', y='feature', palette='viridis')
plt.title('Top 15 Most Important Features (Random Forest)', fontsize=14, fontweight='bold')
plt.xlabel('Feature Importance')
plt.tight_layout()
plt.show()

### 5.5 Gradient Boosting Regressor

In [ ]:
# Train Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
gb_results = evaluate_regression_model('Gradient Boosting', gb_model,
                                      X_train_scaled, y_train,
                                      X_test_scaled, y_test)

### 5.6 XGBoost Regressor

In [ ]:
# Train XGBoost Regressor
xgb_model = XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)
xgb_results = evaluate_regression_model('XGBoost', xgb_model,
                                       X_train_scaled, y_train,
                                       X_test_scaled, y_test)

## 6. Model Comparison and Final Results

In [ ]:
# Compile all results
results_df = pd.DataFrame([lr_results, ridge_results, lasso_results, 
                           rf_results, gb_results, xgb_results])
results_df = results_df.round(4)

print("="*80)
print("MODEL PERFORMANCE COMPARISON")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# R² Score Comparison
axes[0, 0].bar(results_df['model'], results_df['test_r2'], 
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#FFD700'])
axes[0, 0].set_title('Model R² Score Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].set_ylim([0, 1])
for i, v in enumerate(results_df['test_r2']):
    axes[0, 0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

# RMSE Comparison
axes[0, 1].bar(results_df['model'], results_df['test_rmse'],
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#FFD700'])
axes[0, 1].set_title('Model RMSE Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('RMSE ($)')
axes[0, 1].tick_params(axis='x', rotation=45)
for i, v in enumerate(results_df['test_rmse']):
    axes[0, 1].text(i, v + 1, f'${v:.2f}', ha='center', fontweight='bold', fontsize=9)

# MAE Comparison
axes[1, 0].bar(results_df['model'], results_df['test_mae'],
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#FFD700'])
axes[1, 0].set_title('Model MAE Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('MAE ($)')
axes[1, 0].tick_params(axis='x', rotation=45)
for i, v in enumerate(results_df['test_mae']):
    axes[1, 0].text(i, v + 0.5, f'${v:.2f}', ha='center', fontweight='bold', fontsize=9)

# Train vs Test R² (Overfitting check)
x = np.arange(len(results_df))
width = 0.35
axes[1, 1].bar(x - width/2, results_df['train_r2'], width, label='Train R²', alpha=0.8)
axes[1, 1].bar(x + width/2, results_df['test_r2'], width, label='Test R²', alpha=0.8)
axes[1, 1].set_title('Train vs Test R² Score', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(results_df['model'], rotation=45, ha='right')
axes[1, 1].legend()
axes[1, 1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

# Identify best model
best_model_idx = results_df['test_r2'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'model']
best_r2 = results_df.loc[best_model_idx, 'test_r2']
best_rmse = results_df.loc[best_model_idx, 'test_rmse']
best_mae = results_df.loc[best_model_idx, 'test_mae']

print("\n" + "="*80)
print("🏆 BEST MODEL SELECTION")
print("="*80)
print(f"Best Model: {best_model_name}")
print(f"R² Score: {best_r2:.4f}")
print(f"RMSE: ${best_rmse:.2f}")
print(f"MAE: ${best_mae:.2f}")
print("="*80)
print("\n✅ Regression model training and evaluation completed successfully!")

## 7. Conclusions and Key Findings

### Summary of Results:
1. **Objective**: Predict transaction amounts using regression models
2. **Models Tested**: Linear, Ridge, Lasso, Random Forest, Gradient Boosting, XGBoost
3. **Best Model**: Based on R² Score and RMSE

### Key Insights:
- **R² Score**: Indicates how well the model explains variance in transaction amounts
- **RMSE**: Root Mean Squared Error - penalizes large errors more heavily
- **MAE**: Mean Absolute Error - average prediction error in dollars
- Tree-based models (Random Forest, Gradient Boosting, XGBoost) typically perform better than linear models for this complex dataset

### Model Selection Criteria:
- **Highest R² Score**: Better explained variance
- **Lowest RMSE/MAE**: More accurate predictions
- **Train-Test Gap**: Check for overfitting

### Business Applications:
1. **Fraud Detection Support**: Unusual predicted vs actual amounts may indicate fraud
2. **Revenue Forecasting**: Predict future transaction volumes
3. **Risk Assessment**: Identify high-value transactions for additional verification
4. **Customer Behavior**: Understand transaction patterns

### Future Improvements:
1. Feature engineering (time-based features, interaction terms)
2. Ensemble methods combining multiple models
3. Deep learning approaches (Neural Networks)
4. Cross-validation for more robust evaluation
5. Hyperparameter tuning for optimization